### **Customer**


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from typing import List
from pyspark.sql import DataFrame
from pyspark.sql.window import Window


In [0]:
import os
import sys

In [0]:
from custom_utils import transformationss

In [0]:
transformationss

In [0]:
class transformations:
    def dedup(self,df:DataFrame,dedup_cols:list,cdc:str):

        # dedup_list
        df = df.withColumn("dedupKey",concat(*dedup_cols))
        df = df.withColumn("dedupCounts",row_number()\
            .over(Window.partitionBy("dedupKey").orderBy(desc(cdc))))
        df = df.filter(col('dedupCounts')==1)
        df = df.drop('dedupKey','dedupCounts')
        return df

In [0]:
df_cust = spark.read.table("pysparkdbt.bronze.customers")

In [0]:
df_cust = df_cust.withColumn("phone_number", regexp_replace(col("phone_number"), "[^0-9]", ""))
display(df_cust)

In [0]:

df_cust =df_cust.withColumn("full_name",concat_ws(" ",col("first_name"),col("last_name")))
df_cust = df_cust.drop("first_name","last_name")
display(df_cust)

In [0]:
cust_obj = transformations()
df_cust_trns = cust_obj.dedup(df_cust,['customer_id'],"last_updated_timestamp")
display(df_cust_trns)
